In [0]:
display(spark.sql("LIST 'abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/raw'"))

path,name,size,modification_time
abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/raw/online_retail_II.csv,online_retail_II.csv,43954955,1767279960000
abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/raw/online_retail_II.xlsx,online_retail_II.xlsx,45628546,1767266090000


BRONZE LAYER (RAW → DELTA)
📘 Explanation

Read CSV

Store as Delta

No transformations


In [0]:
# ---- Parameters (CI/CD compatible) ----
dbutils.widgets.text("storage_account", "retaildatalaketest")
dbutils.widgets.text("container", "ecommerce")

storage_account = dbutils.widgets.get("storage_account")
container = dbutils.widgets.get("container")

base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net"

In [0]:
raw_path = f"{base_path}/raw/online_retail_II.csv"
bronze_path = f"{base_path}/bronze/online_retail_bronze"
print(bronze_path)

# Read CSV with inferSchema to get proper types where possible
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")   # Important: tries to detect numbers, dates, etc.
      .csv(raw_path)
     )
assert df.count() > 0, "❌ RAW layer is empty"
# Write as Delta in bronze (raw but structured)
df.write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .save(bronze_path)

# Quick check
spark.read.format("delta").load(bronze_path).limit(5).display()

abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/bronze/online_retail_bronze


Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom
